# 라이브러리 로딩

In [1]:
import numpy as np # Numpy
import pandas as pd # Pandas
import matplotlib as mpl #Matplotlib 세팅용
import matplotlib.pyplot as plt # 시각화 도구
import seaborn as sns # 시각화 도구
from sklearn.model_selection import train_test_split # 데이터셋 분리
from sklearn.model_selection import KFold # KFold 교차검증
from sklearn.cluster import KMeans # 클러스터링
from sklearn.metrics import silhouette_score # 실루엣 점수
import xgboost as xgb # XGBoost
from sklearn.model_selection import GridSearchCV # 그리드 서치
from sklearn.metrics import accuracy_score, precision_score # 평가 지표
from sklearn.metrics import recall_score, confusion_matrix, roc_auc_score, f1_score # 평가 지표
from imblearn.combine import SMOTEENN, SMOTETomek # 복합샘플링
from hyperopt import hp, fmin, tpe, Trials # HyperOPT
# 스태킹 앙상블
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
# cv스태밍
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import StratifiedKFold


import warnings # 경고문 제거용


%matplotlib inline
%config Inlinebackend.figure_format = 'retina'

# 한글 폰트 설정
mpl.rc('font', family='D2Coding')
# 유니코드에서 음수 부호 설정
mpl.rc('axes', unicode_minus = False)

warnings.filterwarnings('ignore')
sns.set(font="D2Coding", rc={"axes.unicode_minus":False}, style='whitegrid')
plt.rc('figure', figsize=(10,8))

# 데이터 불러오기

In [2]:
data = pd.read_excel('train_test_na_filled.xlsx', sheet_name='Train')

# 전처리

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8693 non-null   object 
 2   CryoSleep     8693 non-null   bool   
 3   Cabin1        8590 non-null   object 
 4   Cabin2        8590 non-null   float64
 5   Combi         8590 non-null   object 
 6   Cabin3        8590 non-null   object 
 7   Cabin         8590 non-null   object 
 8   Destination   8693 non-null   object 
 9   Age           8693 non-null   int64  
 10  VIP           8693 non-null   bool   
 11  RoomService   8693 non-null   int64  
 12  FoodCourt     8693 non-null   int64  
 13  ShoppingMall  8693 non-null   int64  
 14  Spa           8693 non-null   int64  
 15  VRDeck        8693 non-null   int64  
 16  Name          8493 non-null   object 
 17  Transported   8693 non-null   bool   
dtypes: bool(3), float64(1), int6

## 금액 값 피처 합치기

In [4]:
data['Price'] = data.RoomService + data.FoodCourt + data.ShoppingMall + data.Spa + data.VRDeck
data.Price

0           0
1         736
2       10383
3        5176
4        1091
        ...  
8688     8536
8689        0
8690     1873
8691     4637
8692     4826
Name: Price, Length: 8693, dtype: int64

## 필요없는 features 제거

In [5]:
# 필요없는 features 제거
data.drop(['PassengerId', 'Cabin', 'Combi', 'Name',
           'RoomService','FoodCourt','ShoppingMall','Spa', 'ShoppingMall', 'VRDeck'], axis=1, inplace=True)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   HomePlanet   8693 non-null   object 
 1   CryoSleep    8693 non-null   bool   
 2   Cabin1       8590 non-null   object 
 3   Cabin2       8590 non-null   float64
 4   Cabin3       8590 non-null   object 
 5   Destination  8693 non-null   object 
 6   Age          8693 non-null   int64  
 7   VIP          8693 non-null   bool   
 8   Transported  8693 non-null   bool   
 9   Price        8693 non-null   int64  
dtypes: bool(3), float64(1), int64(2), object(4)
memory usage: 501.0+ KB


## 처리하기 힘든 결측값 제거

In [7]:
data.isna().sum()

HomePlanet       0
CryoSleep        0
Cabin1         103
Cabin2         103
Cabin3         103
Destination      0
Age              0
VIP              0
Transported      0
Price            0
dtype: int64

In [8]:
# 결측값들 제거(Cabin)
data.dropna(axis=0, inplace=True)

## Boolean 캐스팅

In [9]:
# Cabin3의 값을 변환
#data['Cabin3'].replace({'P': True,'S': False}, inplace=True)
#data['Cabin3'] = data['Cabin3'].astype(bool)

## 원핫인코딩

In [10]:
# 원핫인코딩
train_encoding = pd.get_dummies(data['HomePlanet'])
data=data.drop('HomePlanet',axis=1)
data = data.join(train_encoding)

train_encoding = pd.get_dummies(data['Destination'])
data=data.drop('Destination',axis=1)
data = data.join(train_encoding)

train_encoding = pd.get_dummies(data['Cabin1'])
data=data.drop('Cabin1',axis=1)
data = data.join(train_encoding)

train_encoding = pd.get_dummies(data['Cabin3'])
data=data.drop('Cabin3',axis=1)
data = data.join(train_encoding)

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8590 entries, 0 to 8692
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CryoSleep      8590 non-null   bool   
 1   Cabin2         8590 non-null   float64
 2   Age            8590 non-null   int64  
 3   VIP            8590 non-null   bool   
 4   Transported    8590 non-null   bool   
 5   Price          8590 non-null   int64  
 6   Earth          8590 non-null   uint8  
 7   Europa         8590 non-null   uint8  
 8   Mars           8590 non-null   uint8  
 9   55 Cancri e    8590 non-null   uint8  
 10  PSO J318.5-22  8590 non-null   uint8  
 11  TRAPPIST-1e    8590 non-null   uint8  
 12  A              8590 non-null   uint8  
 13  B              8590 non-null   uint8  
 14  C              8590 non-null   uint8  
 15  D              8590 non-null   uint8  
 16  E              8590 non-null   uint8  
 17  F              8590 non-null   uint8  
 18  G       

## 스케일링

In [12]:
# 스케일링
col = ['Cabin2', 'Age', 'Price']
def data_scaled(df, col):
    for i in col:
        data_mean = df[i].mean()
        data_std = df[i].std()
        scaled = (df[i]-data_mean)/data_std
        df[i]=scaled
    return df

In [13]:
data_scaled(data, col)

,CryoSleep,Cabin2,Age,VIP,Transported,Price,Earth,Europa,Mars,55 Cancri e,...,A,B,C,D,E,F,G,T,P,S
0,False,-1.170228,0.712274,False,False,-0.515235,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
1,False,-1.170228,-0.332624,False,True,-0.251820,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
2,False,-1.170228,2.035811,True,False,3.200847,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1
3,False,-1.170228,0.294315,False,False,1.337259,0,1,0,0,...,1,0,0,0,0,0,0,0,0,1
4,False,-1.168274,-0.889902,False,True,-0.124765,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,False,-0.978724,0.851594,True,False,2.539805,0,1,0,1,...,1,0,0,0,0,0,0,0,1,0
8689,True,1.758999,-0.750583,False,False,-0.515235,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
8690,False,1.760953,-0.193304,False,True,0.155113,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
8691,False,0.017878,0.224655,False,False,1.144350,0,1,0,1,...,0,0,0,0,1,0,0,0,0,1


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8590 entries, 0 to 8692
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CryoSleep      8590 non-null   bool   
 1   Cabin2         8590 non-null   float64
 2   Age            8590 non-null   float64
 3   VIP            8590 non-null   bool   
 4   Transported    8590 non-null   bool   
 5   Price          8590 non-null   float64
 6   Earth          8590 non-null   uint8  
 7   Europa         8590 non-null   uint8  
 8   Mars           8590 non-null   uint8  
 9   55 Cancri e    8590 non-null   uint8  
 10  PSO J318.5-22  8590 non-null   uint8  
 11  TRAPPIST-1e    8590 non-null   uint8  
 12  A              8590 non-null   uint8  
 13  B              8590 non-null   uint8  
 14  C              8590 non-null   uint8  
 15  D              8590 non-null   uint8  
 16  E              8590 non-null   uint8  
 17  F              8590 non-null   uint8  
 18  G       

# 평가 함수 정의

In [15]:
def score_funcs(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    roc_auc = roc_auc_score(y_test, pred_proba)
    
    print('오차행렬 \n', confusion)
    print('정확도 : {0:.4f}, 정밀도: {1:.4f}, \
         재현율 : {2:.4f}, F1 : {3:.4f}, AUC : {4:.4f}'.format(accuracy, precision, recall, f1, roc_auc))

# stacking Ensemble

## 기본 스태킹

In [16]:
# 데이터 셋 설정
X_data = data.drop('Transported', axis = 1)
y_label = data['Transported']

# 세트 분리
X_train, X_test, y_train, y_test = train_test_split(X_data, y_label, test_size =0.2, random_state=109)

### 최근접 모델(그리드서치)

In [17]:
knn_clf = KNeighborsClassifier()

params = {
    'n_neighbors':[x for x in range(1, 100)]
}

gsk = GridSearchCV(knn_clf, param_grid = params, cv = 5, refit = True,
                  n_jobs=-1)
gsk.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                         23, 24, 25, 26, 27, 28, 29, 30, ...]})

In [18]:
gsk.best_params_

{'n_neighbors': 18}

In [19]:
tr_knn_pred = gsk.predict(X_train)
tr_knn_proba = gsk.predict_proba(X_train)[:,1]
t_knn_pred = gsk.predict(X_test)
t_knn_proba = gsk.predict_proba(X_test)[:,1]


print('훈련셋 평가 지표')
score_funcs(y_train,tr_knn_pred,tr_knn_proba)
print('\n테스트셋 평가 지표')
score_funcs(y_test,t_knn_pred,t_knn_proba)

훈련셋 평가 지표
오차행렬 
 [[2990  425]
 [1181 2276]]
정확도 : 0.7663, 정밀도: 0.8427,          재현율 : 0.6584, F1 : 0.7392, AUC : 0.8454

테스트셋 평가 지표
오차행렬 
 [[714 128]
 [292 584]]
정확도 : 0.7555, 정밀도: 0.8202,          재현율 : 0.6667, F1 : 0.7355, AUC : 0.8181


### 랜덤포레스트(HyperOPT)

In [20]:
rf_search_space = {'max_depth': hp.quniform('max_depth', 5, 15, 1),
                   'min_samples_split': hp.quniform('min_samples_split', 2, 20, 1),
                   'n_estimators': hp.quniform('n_estimators', 50, 500, 25)}

In [21]:
# fmin()에서 호출 시 search_space 값으로 XGBClassifier 교차 검증 학습 후 -1 * roc_auc 평균 값을 반환
def rf_objective_func(search_space):
    rf = RandomForestClassifier(
        n_estimators=int(search_space['n_estimators']),
                                      max_depth=int(search_space['max_depth']),
                                      min_samples_split=int(search_space['min_samples_split']))
    
    # 3개 k-fold 방식으로 평가된 roc_auc 지표를 담는 list
    roc_auc_list = []
    
    # 3개 k-fold 방식 적용
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=109)
    
    # X_train을 다시 학습과 검증용 데이터로 분리
    for tr_index, val_index in kf.split(X_train, y_train):
        # kf.split(X_train)으로 추출된 학습과 검증 index 값으로 학습과 검증 데이터 세트 분리
        X_tr, y_tr = X_train.iloc[tr_index], y_train.iloc[tr_index]
        X_val, y_val = X_train.iloc[val_index], y_train.iloc[val_index]
        
        # early stopping은 30회로 설정하고 추출된 학습과 검증 데이터로 XGBClassifier 학습 수행
        rf.fit(X_tr, y_tr)
        
        # 1로 예측한 확률값 추출 후 roc auc 계산하고 평균 roc auc 계산을 위해 list에 결과값 담음.
        score = roc_auc_score(y_val, rf.predict_proba(X_val)[:,1])
        roc_auc_list.append(score)
        
    # 3개 k-fold로 계산된 roc_auc 값의 평균값을 반환하되,
    # HyperOPT는 목적함수의 최솟값을 위한 입력값을 찾으므로 -1을 곱한 뒤 반환
    return -1*np.mean(roc_auc_list)

In [22]:
trials = Trials()

# fmin() 함수를 호출. max_evals 지정된 횟수만큼 반복 후 목적함수의 최솟값을 가지는 최적 입력값 추출
best = fmin(fn=rf_objective_func,
           space=rf_search_space,
           algo=tpe.suggest,
           max_evals=50, # 최대 반복 횟수를 지정합니다
           trials=trials, rstate=np.random.default_rng(seed=109))

print('best:', best)

100%|███████████████████████████████████████████████| 50/50 [02:40<00:00,  3.21s/trial, best loss: -0.8215316287066843]
best: {'max_depth': 10.0, 'min_samples_split': 18.0, 'n_estimators': 425.0}


In [23]:
rf = RandomForestClassifier(min_samples_split=19, n_estimators=450, max_depth=12, random_state=109)
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=12, min_samples_split=19, n_estimators=450,
                       random_state=109)

In [24]:
tr_rf_pred = rf.predict(X_train)
tr_rf_proba = rf.predict_proba(X_train)[:,1]
t_rf_pred = rf.predict(X_test)
t_rf_proba = rf.predict_proba(X_test)[:,1]

print('훈련셋 평가 지표')
score_funcs(y_train,tr_rf_pred,tr_rf_proba)
print('\n테스트셋 평가 지표')
score_funcs(y_test,t_rf_pred,t_rf_proba)

훈련셋 평가 지표
오차행렬 
 [[3106  309]
 [ 949 2508]]
정확도 : 0.8169, 정밀도: 0.8903,          재현율 : 0.7255, F1 : 0.7995, AUC : 0.9140

테스트셋 평가 지표
오차행렬 
 [[707 135]
 [276 600]]
정확도 : 0.7608, 정밀도: 0.8163,          재현율 : 0.6849, F1 : 0.7449, AUC : 0.8319


### XGBoost(HyperOPT)

In [25]:
xgb_search_space = {'max_depth': hp.quniform('max_depth', 5, 15, 1),
                   'min_child_weight': hp.quniform('min_child_weight', 1, 20, 1),
                   'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 0.95),
                   'learning_rate': hp.uniform('learning_rate', 0.01, 0.4),
                   'gamma': hp.uniform('gamma', 0, 4),
                   'n_estimators': hp.quniform('n_estimators', 50, 500, 25)}

In [26]:
# fmin()에서 호출 시 search_space 값으로 XGBClassifier 교차 검증 학습 후 -1 * roc_auc 평균 값을 반환
def xgb_objective_func(search_space):
    xgb_clf = xgb.XGBClassifier(n_estimators=int(search_space['n_estimators']),
                                max_depth=int(search_space['max_depth']),
                                min_child_weight=int(search_space['min_child_weight']),
                                colsample_bytree=search_space['colsample_bytree'],
                                learning_rate=search_space['learning_rate'],
                                gamma=search_space['gamma'])
    
    # 3개 k-fold 방식으로 평가된 roc_auc 지표를 담는 list
    roc_auc_list = []
    
    # 3개 k-fold 방식 적용
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=109)
    
    # X_train을 다시 학습과 검증용 데이터로 분리
    for tr_index, val_index in kf.split(X_train, y_train):
        # kf.split(X_train)으로 추출된 학습과 검증 index 값으로 학습과 검증 데이터 세트 분리
        X_tr, y_tr = X_train.iloc[tr_index], y_train.iloc[tr_index]
        X_val, y_val = X_train.iloc[val_index], y_train.iloc[val_index]
        
        # early stopping은 30회로 설정하고 추출된 학습과 검증 데이터로 XGBClassifier 학습 수행
        xgb_clf.fit(X_tr, y_tr, early_stopping_rounds=30, eval_metric="auc",
                   eval_set=[(X_tr, y_tr), (X_val, y_val)])
        
        # 1로 예측한 확률값 추출 후 roc auc 계산하고 평균 roc auc 계산을 위해 list에 결과값 담음.
        score = roc_auc_score(y_val, xgb_clf.predict_proba(X_val)[:,1])
        roc_auc_list.append(score)
        
    # 3개 k-fold로 계산된 roc_auc 값의 평균값을 반환하되,
    # HyperOPT는 목적함수의 최솟값을 위한 입력값을 찾으므로 -1을 곱한 뒤 반환
    return -1*np.mean(roc_auc_list)

In [27]:
trials = Trials()

# fmin() 함수를 호출. max_evals 지정된 횟수만큼 반복 후 목적함수의 최솟값을 가지는 최적 입력값 추출
best = fmin(fn=xgb_objective_func,
           space=xgb_search_space,
           algo=tpe.suggest,
           max_evals=50, # 최대 반복 횟수를 지정합니다
           trials=trials, rstate=np.random.default_rng(seed=109))

print('best:', best)

[0]	validation_0-auc:0.83752	validation_1-auc:0.82342                                                                  
[1]	validation_0-auc:0.84373	validation_1-auc:0.82449                                                                  
[2]	validation_0-auc:0.84996	validation_1-auc:0.82920                                                                  
[3]	validation_0-auc:0.85529	validation_1-auc:0.82887                                                                  
[4]	validation_0-auc:0.85753	validation_1-auc:0.82959                                                                  
[5]	validation_0-auc:0.86374	validation_1-auc:0.82965                                                                  
[6]	validation_0-auc:0.86535	validation_1-auc:0.82979                                                                  
[7]	validation_0-auc:0.86908	validation_1-auc:0.82829                                                                  
[8]	validation_0-auc:0.87367	validation_

[0]	validation_0-auc:0.83695	validation_1-auc:0.80592                                                                  
[1]	validation_0-auc:0.84761	validation_1-auc:0.81325                                                                  
[2]	validation_0-auc:0.85490	validation_1-auc:0.81874                                                                  
[3]	validation_0-auc:0.86016	validation_1-auc:0.81652                                                                  
[4]	validation_0-auc:0.86370	validation_1-auc:0.81820                                                                  
[5]	validation_0-auc:0.86765	validation_1-auc:0.82107                                                                  
[6]	validation_0-auc:0.87054	validation_1-auc:0.82106                                                                  
[7]	validation_0-auc:0.87259	validation_1-auc:0.82281                                                                  
[8]	validation_0-auc:0.87489	validation_

[10]	validation_0-auc:0.86495	validation_1-auc:0.80561                                                                 
[11]	validation_0-auc:0.87248	validation_1-auc:0.80709                                                                 
[12]	validation_0-auc:0.87385	validation_1-auc:0.80630                                                                 
[13]	validation_0-auc:0.87837	validation_1-auc:0.80779                                                                 
[14]	validation_0-auc:0.87804	validation_1-auc:0.80755                                                                 
[15]	validation_0-auc:0.88334	validation_1-auc:0.80654                                                                 
[16]	validation_0-auc:0.88673	validation_1-auc:0.80599                                                                 
[17]	validation_0-auc:0.88958	validation_1-auc:0.80599                                                                 
[18]	validation_0-auc:0.89346	validation

[28]	validation_0-auc:0.90556	validation_1-auc:0.82781                                                                 
[29]	validation_0-auc:0.90631	validation_1-auc:0.82735                                                                 
[30]	validation_0-auc:0.90741	validation_1-auc:0.82721                                                                 
[31]	validation_0-auc:0.90811	validation_1-auc:0.82648                                                                 
[32]	validation_0-auc:0.90835	validation_1-auc:0.82632                                                                 
[33]	validation_0-auc:0.90855	validation_1-auc:0.82639                                                                 
[34]	validation_0-auc:0.90911	validation_1-auc:0.82707                                                                 
[35]	validation_0-auc:0.90911	validation_1-auc:0.82724                                                                 
[36]	validation_0-auc:0.90983	validation

[32]	validation_0-auc:0.86409	validation_1-auc:0.81403                                                                 
[33]	validation_0-auc:0.86499	validation_1-auc:0.81442                                                                 
[34]	validation_0-auc:0.86518	validation_1-auc:0.81481                                                                 
[35]	validation_0-auc:0.86505	validation_1-auc:0.81482                                                                 
[36]	validation_0-auc:0.86574	validation_1-auc:0.81477                                                                 
[37]	validation_0-auc:0.86605	validation_1-auc:0.81461                                                                 
[38]	validation_0-auc:0.86628	validation_1-auc:0.81461                                                                 
[39]	validation_0-auc:0.86698	validation_1-auc:0.81469                                                                 
[40]	validation_0-auc:0.86706	validation

...

[45]	validation_0-auc:0.92696	validation_1-auc:0.81517                                                                 
[46]	validation_0-auc:0.92757	validation_1-auc:0.81459                                                                 
[47]	validation_0-auc:0.92800	validation_1-auc:0.81443                                                                 
[48]	validation_0-auc:0.92800	validation_1-auc:0.81443                                                                 
[49]	validation_0-auc:0.92850	validation_1-auc:0.81375                                                                 
[0]	validation_0-auc:0.76700	validation_1-auc:0.70610                                                                  
[1]	validation_0-auc:0.83497	validation_1-auc:0.79645                                                                  
[2]	validation_0-auc:0.84634	validation_1-auc:0.80960                                                                  
[3]	validation_0-auc:0.84492	validation_

[47]	validation_0-auc:0.91061	validation_1-auc:0.83264                                                                 
[48]	validation_0-auc:0.91061	validation_1-auc:0.83264                                                                 
[49]	validation_0-auc:0.91077	validation_1-auc:0.83277                                                                 
[50]	validation_0-auc:0.91112	validation_1-auc:0.83253                                                                 
[51]	validation_0-auc:0.91112	validation_1-auc:0.83253                                                                 
[52]	validation_0-auc:0.91213	validation_1-auc:0.83208                                                                 
[53]	validation_0-auc:0.91272	validation_1-auc:0.83189                                                                 
[54]	validation_0-auc:0.91329	validation_1-auc:0.83199                                                                 
[55]	validation_0-auc:0.91344	validation

[36]	validation_0-auc:0.90001	validation_1-auc:0.82472                                                                 
[37]	validation_0-auc:0.90055	validation_1-auc:0.82380                                                                 
[38]	validation_0-auc:0.90122	validation_1-auc:0.82351                                                                 
[39]	validation_0-auc:0.90122	validation_1-auc:0.82351                                                                 
[40]	validation_0-auc:0.90122	validation_1-auc:0.82351                                                                 
[41]	validation_0-auc:0.90122	validation_1-auc:0.82351                                                                 
[42]	validation_0-auc:0.90143	validation_1-auc:0.82349                                                                 
[43]	validation_0-auc:0.90231	validation_1-auc:0.82340                                                                 
[0]	validation_0-auc:0.75651	validation_

[13]	validation_0-auc:0.84967	validation_1-auc:0.80674                                                                 
[14]	validation_0-auc:0.84956	validation_1-auc:0.80649                                                                 
[15]	validation_0-auc:0.85240	validation_1-auc:0.80575                                                                 
[16]	validation_0-auc:0.85542	validation_1-auc:0.80526                                                                 
[17]	validation_0-auc:0.85857	validation_1-auc:0.80465                                                                 
[18]	validation_0-auc:0.86029	validation_1-auc:0.80483                                                                 
[19]	validation_0-auc:0.86298	validation_1-auc:0.80563                                                                 
[20]	validation_0-auc:0.86448	validation_1-auc:0.80411                                                                 
[21]	validation_0-auc:0.86705	validation

[22]	validation_0-auc:0.86181	validation_1-auc:0.82610                                                                 
[23]	validation_0-auc:0.86276	validation_1-auc:0.82592                                                                 
[24]	validation_0-auc:0.86481	validation_1-auc:0.82653                                                                 
[25]	validation_0-auc:0.86515	validation_1-auc:0.82607                                                                 
[26]	validation_0-auc:0.86880	validation_1-auc:0.82547                                                                 
[27]	validation_0-auc:0.86980	validation_1-auc:0.82488                                                                 
[28]	validation_0-auc:0.87063	validation_1-auc:0.82530                                                                 
[29]	validation_0-auc:0.87121	validation_1-auc:0.82549                                                                 
[30]	validation_0-auc:0.87121	validation

In [41]:
xgb_clf = xgb.XGBClassifier(colsample_bytree=0.771726554721117, gamma=2.9316238319502728,
                         learning_rate=0.18521774301570038,
                         max_depth=8, min_child_weight=4, random_state=109)
xgb_clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.771726554721117, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=2.9316238319502728, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.18521774301570038, max_bin=256,
              max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
              max_depth=8, max_leaves=0, min_child_weight=4, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=109, ...)

In [42]:
tr_xgb_clf_pred = xgb_clf.predict(X_train)
tr_xgb_clf_proba = xgb_clf.predict_proba(X_train)[:,1]
t_xgb_clf_pred = xgb_clf.predict(X_test)
t_xgb_clf_proba = xgb_clf.predict_proba(X_test)[:,1]

print('훈련셋 평가 지표')
score_funcs(y_train,tr_xgb_clf_pred,tr_xgb_clf_proba)
print('\n테스트셋 평가 지표')
score_funcs(y_test,t_xgb_clf_pred,t_xgb_clf_proba)

훈련셋 평가 지표
오차행렬 
 [[3053  362]
 [ 939 2518]]
정확도 : 0.8107, 정밀도: 0.8743,          재현율 : 0.7284, F1 : 0.7947, AUC : 0.9098

테스트셋 평가 지표
오차행렬 
 [[699 143]
 [275 601]]
정확도 : 0.7567, 정밀도: 0.8078,          재현율 : 0.6861, F1 : 0.7420, AUC : 0.8335


### AdaBoost(HyperOPT)

In [30]:
ada_search_space = {'learning_rate': hp.uniform('learning_rate', 0.01, 0.4),
                   'n_estimators': hp.quniform('n_estimators', 50, 500, 25)}

In [31]:
# fmin()에서 호출 시 search_space 값으로 AdaBoostClassifier 교차 검증 학습 후 -1 * roc_auc 평균 값을 반환
def ada_objective_func(search_space):
    base_model = DecisionTreeClassifier(max_depth = 5)

    ada = AdaBoostClassifier(
        base_estimator = base_model,
        n_estimators=int(search_space['n_estimators']),
        learning_rate=search_space['learning_rate'])

    # 3개 k-fold 방식으로 평가된 roc_auc 지표를 담는 list
    roc_auc_list = []

    # 3개 k-fold 방식 적용
    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=109)

    # X_train을 다시 학습과 검증용 데이터로 분리
    for tr_index, val_index in kf.split(X_train, y_train):
        # kf.split(X_train)으로 추출된 학습과 검증 index 값으로 학습과 검증 데이터 세트 분리
        X_tr, y_tr = X_train.iloc[tr_index], y_train.iloc[tr_index]
        X_val, y_val = X_train.iloc[val_index], y_train.iloc[val_index]

        # early stopping은 30회로 설정하고 추출된 학습과 검증 데이터로 XGBClassifier 학습 수행
        ada.fit(X_tr, y_tr)

        # 1로 예측한 확률값 추출 후 roc auc 계산하고 평균 roc auc 계산을 위해 list에 결과값 담음.
        score = roc_auc_score(y_val, rf.predict_proba(X_val)[:,1])
        roc_auc_list.append(score)
        
    # 3개 k-fold로 계산된 roc_auc 값의 평균값을 반환하되,
    # HyperOPT는 목적함수의 최솟값을 위한 입력값을 찾으므로 -1을 곱한 뒤 반환
    return -1*np.mean(roc_auc_list)

In [32]:
trials = Trials()

# fmin() 함수를 호출. max_evals 지정된 횟수만큼 반복 후 목적함수의 최솟값을 가지는 최적 입력값 추출
best = fmin(fn=ada_objective_func,
           space=ada_search_space,
           algo=tpe.suggest,
           max_evals=50, # 최대 반복 횟수를 지정합니다
           trials=trials, rstate=np.random.default_rng(seed=109))

print('best:', best)

100%|███████████████████████████████████████████████| 50/50 [06:14<00:00,  7.49s/trial, best loss: -0.9138353047524118]
best: {'learning_rate': 0.24335181155558971, 'n_estimators': 100.0}


In [44]:
base_model = DecisionTreeClassifier(max_depth = 5)
ada = AdaBoostClassifier(
            base_estimator = base_model,
            n_estimators=100,
            learning_rate=0.24335181155558971)
ada.fit(X_train, y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   learning_rate=0.24335181155558971, n_estimators=100)

In [46]:
tr_ada_pred = ada.predict(X_train)
tr_ada_proba = ada.predict_proba(X_train)[:,1]
t_ada_pred = ada.predict(X_test)
t_ada_proba = ada.predict_proba(X_test)[:,1]

print('훈련셋 평가 지표')
score_funcs(y_train,tr_ada_pred,tr_ada_proba)
print('\n테스트셋 평가 지표')
score_funcs(y_test,t_ada_pred,t_ada_proba)

훈련셋 평가 지표
오차행렬 
 [[3236  179]
 [ 289 3168]]
정확도 : 0.9319, 정밀도: 0.9465,          재현율 : 0.9164, F1 : 0.9312, AUC : 0.9871

테스트셋 평가 지표
오차행렬 
 [[624 218]
 [253 623]]
정확도 : 0.7258, 정밀도: 0.7408,          재현율 : 0.7112, F1 : 0.7257, AUC : 0.7783


### 스태킹 최종 모델(로지스틱 회귀)

In [47]:
# 스태킹 최종 모델
lr_final = LogisticRegression(solver = 'liblinear')

In [49]:
tr_pred = np.array([tr_knn_pred, tr_rf_pred, tr_xgb_clf_pred, tr_ada_pred])
print(tr_pred.shape)

# 칼럼 레벨로 각 알고리즘의 예측 결과를 피처로 만들기
tr_pred = np.transpose(tr_pred)
print(tr_pred.shape)

(4, 6872)
(6872, 4)


In [50]:
# 최종 메타 모델 학습 및 예측
lr_final.fit(tr_pred, y_train)
tr_final = lr_final.predict(tr_pred)
tr_final_proba = lr_final.predict_proba(tr_pred)[:,1]
print('최종 훈련셋 평가 지표\n')
score_funcs(y_train, tr_final, tr_final_proba)

최종 훈련셋 평가 지표

오차행렬 
 [[3236  179]
 [ 289 3168]]
정확도 : 0.9319, 정밀도: 0.9465,          재현율 : 0.9164, F1 : 0.9312, AUC : 0.9408


In [51]:
t_pred = np.array([t_knn_pred, t_rf_pred, t_xgb_clf_pred, t_ada_pred])
print(t_pred.shape)

# 칼럼 레벨로 각 알고리즘의 예측 결과를 피처로 만들기
t_pred = np.transpose(t_pred)
print(t_pred.shape)

(4, 1718)
(1718, 4)


In [52]:
# 최종 메타 모델 학습 및 예측
lr_final.fit(t_pred, y_test)
t_final = lr_final.predict(t_pred)
t_final_proba = lr_final.predict_proba(t_pred)[:,1]
print('최종 테스트셋 평가 지표\n')
score_funcs(y_test, t_final, t_final_proba)

최종 테스트셋 평가 지표

오차행렬 
 [[690 152]
 [253 623]]
정확도 : 0.7643, 정밀도: 0.8039,          재현율 : 0.7112, F1 : 0.7547, AUC : 0.8028


## cv 세트 기반 스태킹

In [ ]:
# 인덱스 재설정
X_train = X_train.sort_index(ascending=False)
y_train = y_train.sort_index(ascending=False)
X_test = X_test.sort_index(ascending=False)
                             

In [ ]:
# 개별 기반 모델에서 최종 모델이 사용할 학습 검정 데이터 생성 함수
def get_data(model, X_train_n, y_train_n, X_test_n, n_folds):
    # kfold 생성
    kf = KFold(n_splits=n_folds, shuffle=False)
    # 넘파이 배열 초기화
    train_fold_pred = np.zeros((X_train_n.shape[0],1))
    test_pred = np.zeros((X_test_n.shape[0], n_folds))
    print(model.__class__.__name__, 'model 시작')
    
    for fold_counter, (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        # 기반 모델의 학습 예측 폴드 데이터 셋 추출
        print('\t 폴드세트 :', fold_counter, ' model 시작')
        X_tr = X_train_n[train_index]
        y_tr = y_train_n[train_index]
        X_te = X_train_n[valid_index]
        
        # fitting(in foldsets)
        model.fit(X_tr, y_tr)
        # predicting(in foldsets)
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1)
        # predicting((in rawtest_sets))
        test_pred[:,fold_counter] = model.predict(X_test_n)
        
    # make test data using mean of predict data
    test_pred_mean = np.mean(test_pred, axis = 1).reshape(-1,1)
    
    # 생성된 학습, 테스트 데이터 반환
    return train_fold_pred, test_pred_mean

In [ ]:
# 배열로 값 바꾸기
X_train_n = X_train.values
y_train_n = y_train.values
X_test_n = X_test.values

In [ ]:
knn_train, knn_test = get_data(knn_clf, X_train_n, y_train_n, X_test_n, 5)
rf_train, rf_test = get_data(rf_clf, X_train_n, y_train_n, X_test_n, 5)
df_train, df_test = get_data(df_clf, X_train_n, y_train_n, X_test_n, 5)
ada_train, ada_test = get_data(ada_clf, X_train_n, y_train_n, X_test_n, 5)

In [ ]:
final_X_train = np.concatenate((knn_train, rf_train, df_train, ada_train), axis = 1)
final_X_test = np.concatenate((knn_test, rf_test, df_test, ada_test), axis =1)
print( '원본 피처 크기(학습, 테스트)', X_train.shape, X_test.shape)
print( '스태킹 피처 크기(학습, 테스트)', final_X_train.shape, final_X_test.shape)

In [ ]:
# 최종 모델에 학습
lr_final.fit(final_X_train, y_train)
stack_final = lr_final.predict(final_X_test)
stack_final_proba = lr_final.predict_proba(final_X_test)[:,1]
print('최종 메타 모델의 예측 정확도 \n', score_funcs(y_test, stack_final, stack_final_proba))

# 스태킹 앙상블 파라미터 설정